# Etapa I: Coleta de Dados com o Selenium
    Coletar dados de 250 séries com as maiores avaliações do site `https://www.imdb.com/pt/

In [ ]:
import logging
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from concurrent.futures import ThreadPoolExecutor, as_completed
from dataclasses import asdict

# Configuração dos logs
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s",
    datefmt="%H:%M:%S"
)

def scrape_serie(serie_data):
    """Abre um navegador separado e coleta dados de uma série"""
    driver = webdriver.Chrome()
    wait = WebDriverWait(driver, 10)

    try:
        driver.get(serie_data["link"])
        popularidade = wait.until(
            EC.presence_of_element_located((
                By.XPATH,
                '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[2]/div[2]/div/div[3]/a/span/div/div[2]/div[1]'))
).text

        elenco_section = wait.until(
            EC.presence_of_all_elements_located((
                By.XPATH,
                '//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[5]'))
        )
        elenco_principal = [ator.text for ator in elenco_section]

        return {
            "info": serie_data["info"],
            "link": serie_data["link"],
            "popularidade": popularidade,
            "elenco_principal": elenco_principal
        }

    except Exception as e:
        logging.error(f"Erro em {serie_data['link']}: {e}")
        return None
    finally:
        driver.quit()


# ----------------------
# COLETA DOS LINKS
# ----------------------
driver = webdriver.Chrome()
wait = WebDriverWait(driver, 10)

driver.get("https://www.imdb.com/pt/")

menu_icon = wait.until(
    EC.element_to_be_clickable((By.XPATH, '//*[@id="imdbHeader-navDrawerOpen"]'))
)
menu_icon.click()

link_series_bem_avaliadas = wait.until(
    EC.element_to_be_clickable((
        By.XPATH,
        '//*[@id="imdbHeader"]/div/aside[1]/div/div[2]/div/div[2]/div[1]/span/div/div/ul/a[2]'))
)
link_series_bem_avaliadas.click()
time.sleep(2) # esperar todas as 250 series carregarem
series = wait.until(
    EC.presence_of_all_elements_located((
        By.XPATH,
        '//*[@id="__next"]/main/div/div[3]/section/div/div[2]/div/ul/li'))
)

series_data = []
for serie in series:
    try:
        info = serie.text.split("\n")
        link = serie.find_element(By.TAG_NAME, 'a').get_attribute('href')
        series_data.append({"info": info, "link": link})
    except Exception as e:
        logging.warning(f"Erro ao extrair série: {e}")

driver.quit()

# ----------------------
# COLETA EM PARALELO
# ----------------------
logging.info(f"Encontradas {len(series_data)} séries. Iniciando coleta em paralelo...")

start_time = time.time()
dados_completos = []
total = len(series_data)
coletadas = 0

with ThreadPoolExecutor(max_workers=5) as executor:  # até 5 navegadores em paralelo
    futures = [executor.submit(scrape_serie, s) for s in series_data]
    for future in as_completed(futures):
        result = future.result()
        coletadas += 1
        if result:
            dados_completos.append(result)
            logging.info(
                f"Coletada: {result['info'][0]} "
                f"({coletadas}/{total} - {coletadas/total*100:.1f}%)"
            )
        else:
            logging.warning(f"Série falhou ({coletadas}/{total} - {coletadas/total*100:.1f}%)")

end_time = time.time()
logging.info(f"Coleta finalizada em {end_time - start_time:.2f} segundos.")

print(dados_completos)


# Etapa II: Tratamento de Dados
    - Criar funções que possibilitam tratar as listas de dados
    - Criar objetos utilizando essas listas
    - Criar um arquivo Json

In [ ]:
from dataclasses import dataclass
import re
import json


# ==========================
# Classes
# ==========================
@dataclass
class Ator:
    nome: str
    papel: str
    n_ep: int
    
    def to_dict(self):
        return {
            "nome": self.nome,
            "personagem": self.papel,
            "quantidade_episodios": self.n_ep
        }

@dataclass
class Serie:
    nome: str
    ordem: int
    ano_estreia: int
    ano_encerramento: int
    episodios: int
    faixa_etaria: int  # Pode ser número ou "Livre" == 0
    avaliacao: float
    link: str
    popularidade: float
    atores: list

    def to_dict(self):
        # Transforma o objeto em Dicionário
        return {
            "titulo": self.nome,
            "ordem": self.ordem,
            "ano_estreia": self.ano_estreia,
            "ano_encerramento": self.ano_encerramento,
            "episodios": self.episodios,
            "classificacao_indicativa": self.faixa_etaria,
            "nota_imdb": self.avaliacao,
            "link": self.link,
            "popularidade": self.popularidade,
            "elenco_principal": self.atores
        }


# ==========================
# Funções de tratamento
# ==========================
def tratar_string(string):
    # Normaliza traço en dash
    string = string.replace("–", "-")
    # Extrai anos
    anos = re.findall(r"\d{4}", string)
    print('anosssss', anos)
    # Compara se existe ano de extreia e fim
    if len(anos) > 1:
        ano_estreia = anos[0]
        ano_encerramento = int(anos[1])
    else:
        ano_estreia = anos[0]
        ano_encerramento = None
        
    # Inicializa
    episodios = None
    faixa = None

    # Regex: pega número de episódios e a faixa etária
    match = re.search(r"(\d+)\s*episódios\s*(\S+)?", string, re.IGNORECASE)
    if match:
        full_number = match.group(1)
        episodios = int(full_number[-2:])
        faixa_raw = match.group(2)
        # verifica se a faixa é um numero se não for a faixa é "Livre" então recebe zero
        if faixa_raw:
            numeros = re.findall(r"\d+", faixa_raw)  # pega todos os números
            if numeros:
                faixa = int(numeros[0])  # converte o primeiro número que encontrar
            else:
                faixa = 0    # se não tiver número, transforma em zero

    return int(ano_estreia), ano_encerramento, episodios, faixa


def tratar_elenco(texto):
    resultados = []
    if isinstance(texto, list):
        texto = "\n".join(texto)

    padrao = re.compile(
        r"([^\n]+)\n"            # Ator
        r"([^\n]+)\n"            # Personagem 
        r"(\d+)\s*episódio[s]?", # Episódios
        re.IGNORECASE
    )

    for ator, personagem, ep in padrao.findall(texto):
        # adiciona a lista um objeto do tipo Atores
        resultados.append(Ator(
                ator.strip().replace('…', ''),
                personagem.strip(),
                int(ep)).to_dict()
        )
    return resultados


def tratamento_geral(dicionario: dict):
    # --Função para o tratamento geral dos dados-- #
    for key, item in dicionario.items():
        # Trata a primeira parte dos dados: 'info' (nome, ano, ep, faixa e avaliação)
        if key == 'info':
            nome_ordem = item[0].split('.', 1)
            nome = nome_ordem[1].strip()
            ordem = int(nome_ordem[0])
            avaliacao = item[2]
            avaliacao = float(avaliacao.replace(",", "."))
            ano_estreia, ano_encerramento, episodio, faixa = tratar_string(item[1])
            faixa = int(faixa)
            
        # Trata o link
        elif key == 'link':
            link = item

        # Trata a popularidade
        elif key == 'popularidade':
            pop = float(item)

        # Trata a lista de Atores
        elif key == 'elenco_principal':
            lista_de_atores = tratar_elenco(item)
    # Retorna um objeto do tipo serie
    return Serie(
        nome,
        ordem,
        ano_estreia,
        ano_encerramento,
        episodio,
        faixa,
        avaliacao,
        link,
        pop,
        lista_de_atores
    )


def covert_json(lista):
    # Função que converte a lista em dicionario e cria um arquivo json ordenado
    lista_dicts = [s.to_dict() for s in lista]
    ordenado = sorted(lista_dicts, key=lambda x: int(x["ordem"]))
    
    with open("series.json", "w", encoding="utf-8") as f:
        json.dump(ordenado, f, ensure_ascii=False, indent=4)


# ==========================
# Execução
# ==========================
lista_de_obj = list(map(tratamento_geral, dados_completos))
covert_json(lista_de_obj)



# *Bônus*
## Teste com BeautifulSoup
    - Teste sómente com a captura de dados sem tratamento

In [ ]:
import asyncio
import aiohttp
import logging
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

logging.basicConfig(level=logging.INFO, format="%(asctime)s [%(levelname)s] %(message)s", datefmt="%H:%M:%S")

# ---------------------- PEGAR LINKS COM SELENIUM ----------------------
def coletar_links():
    driver = webdriver.Chrome()
    wait = WebDriverWait(driver, 10)

    driver.get("https://www.imdb.com/pt/")

    menu_icon = wait.until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="imdbHeader-navDrawerOpen"]'))
    )
    menu_icon.click()

    link_series_bem_avaliadas = wait.until(
        EC.element_to_be_clickable((
            By.XPATH,
            '//*[@id="imdbHeader"]/div/aside[1]/div/div[2]/div/div[2]/div[1]/span/div/div/ul/a[2]'
        ))
    )
    link_series_bem_avaliadas.click()

    series = wait.until(
        EC.presence_of_all_elements_located((
            By.XPATH,
            '//*[@id="__next"]/main/div/div[3]/section/div/div[2]/div/ul/li'
        ))
    )

    series_data = []
    for serie in series:
        try:
            info = serie.text.split("\n")
            link = serie.find_element(By.TAG_NAME, 'a').get_attribute('href')
            series_data.append({"info": info, "link": link})
        except Exception as e:
            logging.warning(f"Erro ao extrair série: {e}")

    driver.quit()
    return series_data

# ---------------------- FUNÇÃO ASSÍNCRONA DE COLETA ----------------------
async def fetch_and_parse(session, serie_data, idx, total):
    try:
        async with session.get(serie_data["link"]) as response:
            html = await response.text()
            soup = BeautifulSoup(html, "html.parser")

            # Popularidade (tentando pegar o equivalente do XPath usado no Selenium)
            popularidade_tag = soup.select_one('a[href*="ratings"] span')
            popularidade = popularidade_tag.get_text(strip=True) if popularidade_tag else "N/A"

            # Elenco principal
            elenco_section = soup.select("section[data-testid='title-cast'] li.ipc-inline-list__item")
            elenco_principal = [ator.get_text(" ", strip=True) for ator in elenco_section]

            logging.info(f"Coletada {idx}/{total} - {serie_data['info'][0]}")

            return {
                "info": serie_data["info"],
                "link": serie_data["link"],
                "popularidade": popularidade,
                "elenco_principal": elenco_principal
            }
    except Exception as e:
        logging.error(f"Erro em {serie_data['link']}: {e}")
        return None

async def coletar_detalhes(series_data):
    start_time = time.time()
    total = len(series_data)

    async with aiohttp.ClientSession() as session:
        tasks = [fetch_and_parse(session, s, idx + 1, total) for idx, s in enumerate(series_data)]
        results = await asyncio.gather(*tasks)

    elapsed = time.time() - start_time
    logging.info(f"Coleta finalizada em {elapsed:.2f} segundos.")
    return [r for r in results if r]

# ---------------------- EXECUÇÃO NO JUPYTER ----------------------
series_data = coletar_links()
logging.info(f"{len(series_data)} links coletados. Iniciando coleta assíncrona...")

# No Jupyter, usar await
dados_completos = await coletar_detalhes(series_data)

# Resultado final
print(dados_completos)
